In [1]:
## Gostaria de deixar alguns adendos , sobre a tratativa do desafio técnico:
## Toda estrutura foi criada na Nuvem pela ferramenta Google Colab , a extensão de arquivos é .IPYNB
## Onde alguns imports são da própria ferramenta. Não iram executar diretamente no Python.
## O google solicita o authorization code , para montar a partição do Google Drive , para acesso aos arquivos
## Exemplo de chave criado pelo google para acesso e validação ao Google Drive: 4/1gEyw_8bqORDulwdq6fgG4F2d-zaqnehD137OdQEQOyuDHS3oN9HTPg
## O usuario deverá logar no Google / Google drive
## O comando spark-subit não irá se aplicar neste caso, para execução do código.
## Qualquer dúvida , fico a disposição.
## Att. Bruno Milhati Cavallini

In [3]:

# instalação 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [4]:
#Instalação Java-8 , instalação Spark , pacotes findsaprk , collections
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install collections-extended

In [5]:
## Setando variaveis de ambiente 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [6]:
# imports
import findspark
findspark.init()
import pandas as pd
import datetime as dt
import collections
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import col
from pyspark.sql.functions import sum as spark_sum
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql import Row
from six.moves import urllib

# SparkContext
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()




In [7]:
# Leitura de arquivos do Google Drive
# Ao executar o codigo o google solicita o authorization code , para montar a partição do Google Drive 
# Exemplo: 4/tQFvkBi4HJjeLRJMCwMOAkXIGBeMeqmpA15ir6t2i-trbmeMLYZ-lxc 

from google.colab import drive 
from google.colab import files
drive.mount('/content/gdrive')

# carregando CSV
file_local = 'gdrive/My Drive/2018_Yellow_Taxi_Trip_Data.csv'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
# Carregando CSV em Fomato Pandas
data = pd.read_csv('gdrive/My Drive/2018_Yellow_Taxi_Trip_Data.csv')

In [9]:
# Visualização das primeira linhas 
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,06/13/2018 10:21:21 AM,06/13/2018 10:39:36 AM,1,2.20,1,N,237,230,1,13.0,0.0,0.5,1.50,0.0,0.3,15.30
1,2,06/13/2018 10:42:40 AM,06/13/2018 10:47:18 AM,1,0.49,1,N,100,230,2,5.0,0.0,0.5,0.00,0.0,0.3,5.80
2,2,06/13/2018 10:54:04 AM,06/13/2018 11:04:37 AM,1,1.48,1,N,48,162,1,8.5,0.0,0.5,1.50,0.0,0.3,10.80
3,2,06/13/2018 10:02:47 AM,06/13/2018 10:27:58 AM,1,2.29,1,N,164,237,1,15.5,0.0,0.5,4.08,0.0,0.3,20.38
4,2,06/13/2018 10:29:11 AM,06/13/2018 10:45:29 AM,1,0.94,1,N,43,161,2,11.0,0.0,0.5,0.00,0.0,0.3,11.80


In [10]:
# Conversão de String para Datetime
data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'])
data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'])

In [11]:
# Visualização das primeira linhas 
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2018-06-13 10:21:21,2018-06-13 10:39:36,1,2.20,1,N,237,230,1,13.0,0.0,0.5,1.50,0.0,0.3,15.30
1,2,2018-06-13 10:42:40,2018-06-13 10:47:18,1,0.49,1,N,100,230,2,5.0,0.0,0.5,0.00,0.0,0.3,5.80
2,2,2018-06-13 10:54:04,2018-06-13 11:04:37,1,1.48,1,N,48,162,1,8.5,0.0,0.5,1.50,0.0,0.3,10.80
3,2,2018-06-13 10:02:47,2018-06-13 10:27:58,1,2.29,1,N,164,237,1,15.5,0.0,0.5,4.08,0.0,0.3,20.38
4,2,2018-06-13 10:29:11,2018-06-13 10:45:29,1,0.94,1,N,43,161,2,11.0,0.0,0.5,0.00,0.0,0.3,11.80


In [12]:
# Aplicando Novas Colunas com dias da semana
data['day_pickup'] = data['tpep_pickup_datetime'].dt.day_name()
data['day_dropoff'] = data['tpep_dropoff_datetime'].dt.day_name()

In [13]:
# Validando coluna 
data['day_pickup'].head()

0    Wednesday
1    Wednesday
2    Wednesday
3    Wednesday
4    Wednesday
Name: day_pickup, dtype: object

In [14]:
# Validando coluna 
data['day_dropoff']

0          Wednesday
1          Wednesday
2          Wednesday
3          Wednesday
4          Wednesday
             ...    
1515289       Monday
1515290       Monday
1515291       Monday
1515292       Monday
1515293       Monday
Name: day_dropoff, Length: 1515294, dtype: object

In [15]:
# inserindo um ID único no modelo
data["id"] = data.index + 1

In [16]:
# Rdd Texto
##linesj = sc.textFile(file_loc1)

# Convertendoum modelo pandas para DataFrame Spark
df = spark.createDataFrame(data)


In [17]:
##df_orders = spark.read.parquet('/dbfs/FileStore/refinado/orders')



##df = spark.read.csv(file_loc1,header=True)
# verificando a Tipagem de dados
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- day_pickup: string (nullable = true)
 |-- day_dropoff: string (nullable = true)
 |-- id: long (nullable = true)



In [18]:
# Criando Tabela temporária 
df.createOrReplaceTempView("TABELA")

In [19]:
# Média de Distancia entre viagens, com no maximo 2 passageiros
query1 = spark.sql("""
select  AVG(trip_distance) trip_distance from TABELA
where passenger_count <= '2'
""").show()

+------------------+
|     trip_distance|
+------------------+
|3.0493525601413425|
+------------------+



In [20]:
# query para verificação dos dias da semana 
spark.sql("""
select  
day_pickup
FROM TABELA
GROUP BY day_pickup
ORDER BY 1
""").show()


+----------+
|day_pickup|
+----------+
|    Friday|
|    Monday|
|  Saturday|
|    Sunday|
|  Thursday|
|   Tuesday|
| Wednesday|
+----------+



In [21]:
# Média de Corridas iniciadas nos finals de semana levando em consideração o campo VendorID
query2= spark.sql("""
select  
AVG(VendorID) MEDIA , day_pickup
FROM TABELA
WHERE day_pickup BETWEEN "Saturday" and "Sunday"
GROUP BY day_pickup
""").show()

+------------------+----------+
|             MEDIA|day_pickup|
+------------------+----------+
|1.5779342246454569|  Saturday|
|1.5933597737368521|    Sunday|
+------------------+----------+



In [22]:
# Notei que o VendorID não é um ID único
spark.sql("""
select  
count(VendorID) QTD, VendorID
FROM TABELA
WHERE day_pickup BETWEEN "Saturday" and "Sunday"
GROUP BY VendorID
""").show()

+------+--------+
|   QTD|VendorID|
+------+--------+
|217281|       1|
|306386|       2|
|    34|       4|
+------+--------+



In [25]:
# Média de Corridas iniciadas nos finals de semana por ID Único
query3 = spark.sql("""
select  
AVG(id) MEDIA , day_pickup
FROM TABELA
WHERE day_pickup BETWEEN "Saturday" and "Sunday"
GROUP BY day_pickup
""").show()

+------------------+----------+
|             MEDIA|day_pickup|
+------------------+----------+
| 963781.1251305067|  Saturday|
|1225600.0986955978|    Sunday|
+------------------+----------+

